In [1]:
import numpy as np
piece_initial_positions = np.array([
        [[1 , 2 , 3 ],
         [4 , 5 , 6 ],
         [7 , 8 , 9 ]], # Front face

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]], # Middle slice

        [[19, 20, 21],
         [22, 23, 24],
         [25, 26, 27]], # Back face
    ])

cube_faces = {
        'X': piece_initial_positions[:,:,2],
        'x': piece_initial_positions[:,:,0],
        'Y': piece_initial_positions[2,:,:],
        'y': piece_initial_positions[0,:,:],
        'Z': piece_initial_positions[:,0,:],
        'z': piece_initial_positions[:,2,:]
}
piece_id = 1
cc = []
for normal in ['X','x','Y','y','Z','z']:
    if piece_id in cube_faces[normal]:
        cc.append(normal)
print(cc)

['x', 'y', 'Z']


In [ ]:
class CubeVisualizer:
    def __init__(self, cube_tracker):
        self.cube_tracker = cube_tracker
        self.output = Output()
        
        # Define color scheme for cube faces
        self.color_map = {
            'W': '#FFFFFF',  # White (top)
            'Y': '#FFFF00',  # Yellow (bottom)
            'O': '#FF8C00',  # Orange (left)
            'R': '#FF0000',  # Red (right)
            'G': '#00FF00',  # Green (front)
            'B': '#0000FF',  # Blue (back)
            'X': '#333333',  # Dark gray (inner faces)
        }

        self.initial_color_vs_direction_map = {
            'x': 'O',
            'X': 'R',
            'y': 'G',
            'Y': 'B',
            'z': 'Y',
            'Z': 'W'
        }

        self.material_index = {
            'X': 0, 
            'x': 1, 
            'Z': 2, 
            'z': 3, 
            'y': 4, 
            'Y': 5
        }
        
        # Set up the 3D scene
        self.scene = Scene(background='#F0F0F0')
        self.camera = PerspectiveCamera(
            position=[6, 6, 6], 
            up=[0, 1, 0],
            aspect=1.0,
            fov=35,
            # Make sure if rotation order is set, it uses uppercase
            rotation_order='XYZ'  # not 'xyz'
        )
        
        # Add lighting
        self.setup_lighting()
        
        # Create a group to hold all cube pieces
        self.cube_group = Group()
        self.scene.add(self.cube_group)
        
        # Create the cube pieces
        self.piece_meshes = {}
        self.create_cube_pieces()
        
        # Renderer with orbit controls
        self.renderer = Renderer(
            camera=self.camera,
            scene=self.scene,
            controls=[OrbitControls(controlling=self.camera)],
            width=450,
            height=450,
            antialias=True
        )
        
        # Animation properties
        self.animation_speed = 0.3  # seconds per move
        self.currently_animating = False
        self.move_history = []
        self.max_history = 30
        
        # Create UI controls
        self.create_ui_controls()
    
    def setup_lighting(self):
        """Set up lighting for better 3D rendering"""
        # Ambient light for overall illumination
        ambient = AmbientLight(color='#ffffff', intensity=0.6)
        self.scene.add(ambient)
        
        # Directional lights from different angles for depth
        light_positions = [
            (5, 10, 5),   # Top-front-right
            (-5, 10, -5), # Top-back-left
            (0, -10, 0),  # Bottom
            (10, 0, 0)    # Right
        ]
        
        for position in light_positions:
            light = DirectionalLight(color='#ffffff', position=position, intensity=0.5)
            self.scene.add(light)
    
    def create_cube_pieces(self):
        """Create 3D meshes for each piece of the cube"""
        # Size of each cubelet (slightly smaller than 1 for visual gap)
        size = 0.95
        gap = (1 - size) / 2
        
        # Clear existing pieces
        for child in list(self.cube_group.children):
            self.cube_group.remove(child)
        
        self.piece_meshes = {}
        
        # Iterate through all positions in the cube
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = self.cube_tracker.piece_current_positions[i, j, k]
                    if piece_id == 14:  # Skip the center piece (optional)
                        continue
                    
                    # Position in 3D space (centered around origin)
                    # Maps cube position to 3D coordinates with origin at cube center
                    x = k - 1
                    y = 1 - j
                    z = 1 - i
                    
                    # Create piece mesh
                    piece_group = self.create_piece_mesh(piece_id, (i, j, k))
                    piece_group.position = [x, y, z]
                    
                    # Store the mesh for future reference
                    self.piece_meshes[piece_id] = {
                        'mesh': piece_group,
                        'position': (i, j, k)
                    }
                    
                    # Add to the scene
                    self.cube_group.add(piece_group)
    
    def create_piece_mesh(self, piece_id, position):
        """Create a mesh for a single cube piece with correct colors"""
        size = 0.9  # Slightly smaller for visual gap between pieces
        piece_group = Group()
        
        # Default material for inner sides (dark gray)
        default_material = MeshLambertMaterial(color=self.color_map['X'])
        
        # Materials for each face of the cube piece (right, left, top, bottom, front, back)
        materials = [default_material] * 6
        
        # Map position to face colors
        i, j, k = position
        if k == 2:  # Right face
            materials[0] = MeshLambertMaterial(color=self.color_map['R'])
        if k == 0:  # Left face
            materials[1] = MeshLambertMaterial(color=self.color_map['O'])
        if j == 0:  # Top face
            materials[2] = MeshLambertMaterial(color=self.color_map['W'])
        if j == 2:  # Bottom face
            materials[3] = MeshLambertMaterial(color=self.color_map['Y'])
        if i == 0:  # Front face
            materials[4] = MeshLambertMaterial(color=self.color_map['G'])
        if i == 2:  # Back face
            materials[5] = MeshLambertMaterial(color=self.color_map['B'])
        
        # Create the box geometry
        geometry = BoxGeometry(size, size, size)
        mesh = Mesh(geometry=geometry, material=materials)
        piece_group.add(mesh)
        
        return piece_group
    
    def update_visualization(self, move):
        """Update the visualization based on current cube state"""
        # For each position in the cube
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = self.cube_tracker.piece_initial_positions[i, j, k]
                    if piece_id == 14:  # Skip the center piece
                        continue
                    
                    # Calculate 3D position
                    x = k - 1
                    y = 1 - j
                    z = 1 - i
                    
                    # Update position
                    if piece_id in self.piece_meshes:
                        self.piece_meshes[piece_id]['position'] = (i, j, k)
                        piece_mesh = self.piece_meshes[piece_id]['mesh']
                        piece_mesh.position = [x, y, z]
                        
                        # Update colors based on orientation and position
                        self.update_piece_colors(piece_id, (i, j, k), move)
        
    def update_piece_colors(self, piece_id, position, move):
        """Update the colors of a piece based on its orientation and position"""
        current_orientation = list(self.cube_tracker.piece_current_orientations[position[0], position[1], position[2]])
        piece_mesh = self.piece_meshes[piece_id]['mesh']
       
        # Get the mesh from the piece group
        if len(piece_mesh.children) > 0:
            cube_mesh = piece_mesh.children[0]
        else:
            return  # No mesh to update
        
        if piece_id in self.cube_tracker.corner_ids:
            cube_faces = CubeBase.cube_faces
            corner_colors = []
            corner_initial_orientation = list(self.cube_tracker.piece_initial_orientations[position[0], position[1], position[2]])

            print("piece_id:", piece_id)
            print("piece_position:", position)
            print("corner_initial_orientation:",corner_initial_orientation)

            for facelet_id in corner_initial_orientation:
                corner_colors.append(self.initial_color_vs_direction_map[facelet_id])

            print("corner_colors:",corner_colors)

            color_vs_ornt = list(zip(corner_initial_orientation, corner_colors))

            print("color_vs_ornt:", color_vs_ornt)
            print("current_orientation:", current_orientation)

            final_colors = []
            for u in current_orientation:
                for v in color_vs_ornt:
                    if u.lower() == v[0].lower():
                        final_colors.append(v[1])

            print("final_colors:", final_colors)

            default_material = MeshLambertMaterial(color=self.color_map['X'])
            materials = [default_material] * 6

            final_corner_facelet_abs_ids = []
            p,q,r  = self.cube_tracker.movements[move][position]
            print("abc:",p,q,r)
            final_pos_piece_id = self.cube_tracker.piece_initial_positions[p,q,r]
            print("final_pos_piece_id", final_pos_piece_id)
            for normal in ['X','x','Y','y','Z','z']:
                if final_pos_piece_id in cube_faces[normal]:
                    final_corner_facelet_abs_ids.append(normal)
            
            print("final_corner_facelet_abs_ids:", final_corner_facelet_abs_ids)
                    
            for idx in range(3):
                materials[self.material_index[final_corner_facelet_abs_ids[idx]]] = MeshLambertMaterial(color=self.color_map[final_colors[idx]])
            cube_mesh.material = materials
            print(cube_mesh.material)

    def get_affected_pieces(self, move):
        """Determine which pieces are affected by a given move"""
        # Based on the move, determine which pieces will be rotated
        return list(self.cube_tracker.movements[move].keys())
    
    def get_move_rotation_params(self, move):
        """Get rotation axis and angle for a move"""
        # Define the rotation parameters for each move
        # (axis, angle in radians)
        rotation_params = {
            'L': ([1, 0, 0], -np.pi/2),
            'l': ([1, 0, 0], np.pi/2),
            'R': ([1, 0, 0], np.pi/2),
            'r': ([1, 0, 0], -np.pi/2),
            'F': ([0, 0, 1], -np.pi/2),
            'f': ([0, 0, 1], np.pi/2),
            'B': ([0, 0, 1], np.pi/2),
            'b': ([0, 0, 1], -np.pi/2),
            'U': ([0, 1, 0], -np.pi/2),
            'u': ([0, 1, 0], np.pi/2),
            'D': ([0, 1, 0], np.pi/2),
            'd': ([0, 1, 0], -np.pi/2)
        }
        
        return rotation_params.get(move, ([0, 0, 0], 0))
    
    def animate_move(self, move):
        """Animate a single move with smooth rotation"""
        if self.currently_animating:
            return
            
        self.currently_animating = True
        
        # Update move history
        self.move_history.append(move)
        if len(self.move_history) > self.max_history:
            self.move_history.pop(0)
        self.update_history_display()
        
        # Helper function to convert axis-angle to quaternion representation
        def axis_angle_to_quaternion(axis, angle):
            """Convert axis-angle rotation to quaternion [x,y,z,w]"""
            # Normalize the axis
            axis_norm = np.sqrt(sum(x*x for x in axis))
            if axis_norm > 0:
                x, y, z = [x/axis_norm for x in axis]
            else:
                return [0, 0, 0, 1]  # Identity quaternion for zero rotation
                
            # Calculate quaternion components
            half_angle = angle / 2
            sin_half = np.sin(half_angle)
            
            return [
                x * sin_half,   # qx
                y * sin_half,   # qy
                z * sin_half,   # qz
                np.cos(half_angle)  # qw
            ]
        
        # Set up animation in a separate thread to keep UI responsive
        def animation_thread():
            try:
                # Get rotation parameters for the move
                axis, angle = self.get_move_rotation_params(move)
                
                # Get affected pieces
                affected_positions = self.get_affected_pieces(move)
                pieces_to_rotate = []
                
                # Gather affected piece meshes
                for pos in affected_positions:
                    i, j, k = pos
                    if i < 3 and j < 3 and k < 3:  # Ensure indices are in range
                        piece_id = self.cube_tracker.piece_current_positions[i, j, k]
                        if piece_id != 14 and piece_id in self.piece_meshes:
                            pieces_to_rotate.append(self.piece_meshes[piece_id]['mesh'])
                
                # Create a temporary group for rotation
                rotation_group = Group()
                self.scene.add(rotation_group)
                
                # Move pieces to the rotation group
                for piece in pieces_to_rotate:
                    if piece in self.cube_group.children:
                        self.cube_group.remove(piece)
                    rotation_group.add(piece)
                
                # Animate the rotation using quaternions
                steps = 50
                for step in range(steps + 1):
                    rotation_angle = angle * step / steps
                    quaternion = axis_angle_to_quaternion(axis, rotation_angle)
                    rotation_group.quaternion = quaternion
                    time.sleep(self.animation_speed / steps)
                
                # Apply the move to the tracker
                self.cube_tracker.apply_moves(move)
                
                # Move pieces back to the cube group
                for piece in list(rotation_group.children):
                    rotation_group.remove(piece)
                    self.cube_group.add(piece)
                
                # Clean up
                self.scene.remove(rotation_group)
                self.update_visualization(move)
            except Exception as e:
                with self.output:
                    print(f"Animation error: {e}")
            finally:
                self.currently_animating = False
        
        # Start animation thread
        threading.Thread(target=animation_thread).start()
    
    def update_history_display(self):
        """Update the move history display"""
        formatted_history = []
        for move in self.move_history:
            if move.islower():
                formatted_history.append(f"{move.upper()}'")
            else:
                formatted_history.append(move)
        
        history_str = " ".join(formatted_history[-20:])  # Show last 20 moves
        with self.output:
            self.history_display.value = f"<b>Move History:</b> {history_str}"
    
    def create_ui_controls(self):
        """Create UI controls for interacting with the cube"""
        # Move buttons
        self.move_buttons = {}
        button_layout = Layout(width='50px', margin='2px')
        
        for move in ['L', 'R', 'F', 'B', 'U', 'D']:
            # Clockwise button
            cw_button = Button(description=move, layout=button_layout, 
                               button_style='primary')
            cw_button.on_click(lambda b, m=move: self.handle_move(m))
            
            # Counter-clockwise button
            ccw_button = Button(description=f"{move}'", layout=button_layout, 
                               button_style='info')
            ccw_button.on_click(lambda b, m=move.lower(): self.handle_move(m))
            
            self.move_buttons[move] = (cw_button, ccw_button)
        
        # Utility buttons
        self.scramble_button = Button(
            description='Scramble', 
            layout=Layout(width='100px', margin='2px'),
            button_style='warning'
        )
        self.scramble_button.on_click(self.handle_scramble)
        
        self.reset_button = Button(
            description='Reset', 
            layout=Layout(width='100px', margin='2px'),
            button_style='danger'
        )
        self.reset_button.on_click(self.handle_reset)
        
        # Algorithm sequence buttons
        self.sequence_buttons = {}
        seq_layout = Layout(width='100px', margin='2px')
        
        common_sequences = {
            'Sexy Move': 'RUru',
            'T Perm': 'RUrFRRuRUFRF',
            'Y Perm': 'FRUruRFrFUrFRuRF'
        }
        
        for name, sequence in common_sequences.items():
            btn = Button(description=name, layout=seq_layout, button_style='success')
            btn.on_click(lambda b, seq=sequence: self.handle_sequence(seq))
            self.sequence_buttons[name] = btn
        
        # Animation speed control
        self.speed_slider = FloatSlider(
            value=0.3,
            min=0.1,
            max=1.0,
            step=0.1,
            description='Speed:',
            continuous_update=False,
            layout=Layout(width='250px')
        )
        self.speed_slider.observe(self.update_speed, names='value')
        
        # Move history display
        self.history_display = HTML(value="<b>Move History:</b>")
        
        # Arrange UI controls
        basic_moves = [
            HBox([self.move_buttons[m][0], self.move_buttons[m][1]]) 
            for m in ['L', 'R', 'F', 'B', 'U', 'D']
        ]
        
        sequences = HBox([self.sequence_buttons[name] for name in common_sequences])
        utilities = HBox([self.scramble_button, self.reset_button])
        
        # Main control panel
        self.controls = VBox([
            HTML("<h3>Rubik's Cube Simulator</h3>"),
            HTML("<b>Basic Moves:</b>"),
            HBox([VBox(basic_moves[:2]), VBox(basic_moves[2:4]), VBox(basic_moves[4:])]),
            HTML("<b>Common Algorithms:</b>"),
            sequences,
            HTML("<b>Utilities:</b>"),
            utilities,
            self.speed_slider,
            self.history_display,
            self.output
        ])
    
    def update_speed(self, change):
        """Update animation speed from slider"""
        self.animation_speed = change.new
    
    def handle_move(self, move):
        """Handle a move button click"""
        if not self.currently_animating:
            self.animate_move(move)
            pass
    
    def handle_sequence(self, sequence):
        """Apply a sequence of moves"""
        if self.currently_animating:
            return
        
        def sequence_thread():
            for move in sequence:
                self.handle_move(move)
                # Wait for animation to complete
                while self.currently_animating:
                    time.sleep(0.05)
                time.sleep(0.1)  # Small pause between moves
        
        threading.Thread(target=sequence_thread).start()
    
    def handle_scramble(self, b):
        """Scramble the cube with random moves"""
        if self.currently_animating:
            return
            
        def scramble_thread():
            try:
                # Generate random sequence of 20 moves
                possible_moves = list('LRFBUDlrfbud')
                moves = [np.random.choice(possible_moves) for _ in range(20)]
                
                with self.output:
                    print(f"Scrambling with sequence: {''.join(moves)}")
                
                for move in moves:
                    # Wait if currently animating
                    while self.currently_animating:
                        time.sleep(0.1)
                    
                    # Apply next move
                    self.animate_move(move)
                    
                    # Wait for animation to complete
                    time.sleep(self.animation_speed + 0.1)
            except Exception as e:
                with self.output:
                    print(f"Scramble error: {e}")
        
        threading.Thread(target=scramble_thread).start()
    
    def handle_reset(self, b):
        """Reset the cube to solved state"""
        if self.currently_animating:
            return
        
        # Create a new cube tracker to reset to initial state
        self.cube_tracker = CubeTracker()
        self.move_history = []
        self.update_history_display()
        
        # Update visualization
        self.create_cube_pieces()
    
    def display(self):
        """Return the widget for display"""
        return VBox([
            self.renderer,
            self.controls
        ])


def create_rubiks_cube_visualization():
    """Create and return the Rubik's cube visualization widget"""
    cube_tracker = CubeTracker()
    visualizer = CubeVisualizer(cube_tracker)
    return visualizer.display()

In [ ]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Layout, HTML, FloatSlider, Output
from pythreejs import (
    BoxGeometry, Mesh, MeshLambertMaterial, Scene, PerspectiveCamera,
    DirectionalLight, AmbientLight, OrbitControls, Renderer, Group
)
import time
import threading

class CubeVisualizer:
    def __init__(self, cube_tracker):
        self.cube_tracker = cube_tracker
        self.output = Output()
        
        # Define color scheme for cube faces
        self.color_map = {
            'W': '#FFFFFF',  # White (top)
            'Y': '#FFFF00',  # Yellow (bottom)
            'O': '#FF8C00',  # Orange (left)
            'R': '#FF0000',  # Red (right)
            'G': '#00FF00',  # Green (front)
            'B': '#0000FF',  # Blue (back)
            'X': '#333333',  # Dark gray (inner faces)
        }
        
        # Set up the 3D scene
        self.scene = Scene(background='#F0F0F0')
        self.camera = PerspectiveCamera(
            position=[6, 6, 6], 
            up=[0, 1, 0],
            aspect=1.0,
            fov=35
        )
        
        # Add lighting
        self.setup_lighting()
        
        # Create a group to hold all cube pieces
        self.cube_group = Group()
        self.scene.add(self.cube_group)
        
        # Create the cube pieces
        self.piece_meshes = {}
        self.create_cube_pieces()
        
        # Renderer with orbit controls
        self.renderer = Renderer(
            camera=self.camera,
            scene=self.scene,
            controls=[OrbitControls(controlling=self.camera)],
            width=450,
            height=450,
            antialias=True
        )
        
        # Animation properties
        self.animation_speed = 0.3  # seconds per move
        self.currently_animating = False
        self.move_history = []
        self.max_history = 30
        
        # Create UI controls
        self.create_ui_controls()
    
    def setup_lighting(self):
        """Set up lighting for better 3D rendering"""
        # Ambient light for overall illumination
        ambient = AmbientLight(color='#ffffff', intensity=0.6)
        self.scene.add(ambient)
        
        # Directional lights from different angles for depth
        light_positions = [
            (5, 10, 5),   # Top-front-right
            (-5, 10, -5), # Top-back-left
            (0, -10, 0),  # Bottom
            (10, 0, 0)    # Right
        ]
        
        for position in light_positions:
            light = DirectionalLight(color='#ffffff', position=position, intensity=0.5)
            self.scene.add(light)
    
    def create_cube_pieces(self):
        """Create 3D meshes for each piece of the cube"""
        # Size of each cubelet (slightly smaller than 1 for visual gap)
        size = 0.95
        gap = (1 - size) / 2
        
        # Clear existing pieces
        for child in list(self.cube_group.children):
            self.cube_group.remove(child)
        
        self.piece_meshes = {}
        
        # Iterate through all positions in the cube
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = self.cube_tracker.piece_current_positions[i, j, k]
                    if piece_id == 14:  # Skip the center piece (optional)
                        continue
                    
                    # Position in 3D space (centered around origin)
                    # Maps cube position to 3D coordinates with origin at cube center
                    x = k - 1
                    y = 1 - j
                    z = 1 - i
                    
                    # Create piece mesh
                    piece_group = self.create_piece_mesh(piece_id, (i, j, k))
                    piece_group.position = [x, y, z]
                    
                    # Store the mesh for future reference
                    self.piece_meshes[piece_id] = {
                        'mesh': piece_group,
                        'position': (i, j, k)
                    }
                    
                    # Add to the scene
                    self.cube_group.add(piece_group)
    
    def create_piece_mesh(self, piece_id, position):
        """Create a mesh for a single cube piece with correct colors"""
        size = 0.9  # Slightly smaller for visual gap between pieces
        piece_group = Group()
        
        # Get the color information for this piece
        color_info = self.cube_tracker.initial_color_map[piece_id]
        
        # Default material for inner sides (dark gray)
        default_material = MeshLambertMaterial(color=self.color_map['X'])
        
        # Materials for each face of the cube piece (right, left, top, bottom, front, back)
        materials = [default_material] * 6
        
        # Map position to face colors
        i, j, k = position
        if k == 2:  # Right face
            materials[0] = MeshLambertMaterial(color=self.color_map['R'])
        if k == 0:  # Left face
            materials[1] = MeshLambertMaterial(color=self.color_map['O'])
        if j == 0:  # Top face
            materials[2] = MeshLambertMaterial(color=self.color_map['W'])
        if j == 2:  # Bottom face
            materials[3] = MeshLambertMaterial(color=self.color_map['Y'])
        if i == 0:  # Front face
            materials[4] = MeshLambertMaterial(color=self.color_map['G'])
        if i == 2:  # Back face
            materials[5] = MeshLambertMaterial(color=self.color_map['B'])
        
        # Create the box geometry
        geometry = BoxGeometry(size, size, size)
        mesh = Mesh(geometry=geometry, material=materials)
        piece_group.add(mesh)
        
        return piece_group
    
    def update_visualization(self):
        """Update the visualization based on current cube state"""
        # Update positions of all pieces
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = self.cube_tracker.piece_current_positions[i, j, k]
                    if piece_id == 14:  # Skip the center piece
                        continue
                    
                    if piece_id in self.piece_meshes:
                        # Calculate 3D position
                        x = k - 1
                        y = 1 - j
                        z = 1 - i
                        
                        # Update position
                        self.piece_meshes[piece_id]['position'] = (i, j, k)
                        self.piece_meshes[piece_id]['mesh'].position = [x, y, z]
    
    def get_affected_pieces(self, move):
        """Determine which pieces are affected by a given move"""
        # Based on the move, determine which pieces will be rotated
        if move in ['L', 'l']:
            return [(i, j, 0) for i in range(3) for j in range(3)]
        elif move in ['R', 'r']:
            return [(i, j, 2) for i in range(3) for j in range(3)]
        elif move in ['F', 'f']:
            return [(0, j, k) for j in range(3) for k in range(3)]
        elif move in ['B', 'b']:
            return [(2, j, k) for j in range(3) for k in range(3)]
        elif move in ['U', 'u']:
            return [(i, 0, k) for i in range(3) for k in range(3)]
        elif move in ['D', 'd']:
            return [(i, 2, k) for i in range(3) for k in range(3)]
        return []
    
    def get_move_rotation_params(self, move):
        """Get rotation axis and angle for a move"""
        # Define the rotation parameters for each move
        # (axis, angle in radians)
        rotation_params = {
            'L': ([1, 0, 0], -np.pi/2),
            'l': ([1, 0, 0], np.pi/2),
            'R': ([1, 0, 0], np.pi/2),
            'r': ([1, 0, 0], -np.pi/2),
            'F': ([0, 0, 1], -np.pi/2),
            'f': ([0, 0, 1], np.pi/2),
            'B': ([0, 0, 1], np.pi/2),
            'b': ([0, 0, 1], -np.pi/2),
            'U': ([0, 1, 0], -np.pi/2),
            'u': ([0, 1, 0], np.pi/2),
            'D': ([0, 1, 0], np.pi/2),
            'd': ([0, 1, 0], -np.pi/2)
        }
        
        return rotation_params.get(move, ([0, 0, 0], 0))
    
    def animate_move(self, move):
        """Animate a single move with smooth rotation"""
        if self.currently_animating:
            return
            
        self.currently_animating = True
        
        # Update move history
        self.move_history.append(move)
        if len(self.move_history) > self.max_history:
            self.move_history.pop(0)
        self.update_history_display()
        
        # Helper function to convert axis-angle to quaternion representation
        def axis_angle_to_quaternion(axis, angle):
            """Convert axis-angle rotation to quaternion [x,y,z,w]"""
            # Normalize the axis
            axis_norm = np.sqrt(sum(x*x for x in axis))
            if axis_norm > 0:
                x, y, z = [x/axis_norm for x in axis]
            else:
                return [0, 0, 0, 1]  # Identity quaternion for zero rotation
                
            # Calculate quaternion components
            half_angle = angle / 2
            sin_half = np.sin(half_angle)
            
            return [
                x * sin_half,   # qx
                y * sin_half,   # qy
                z * sin_half,   # qz
                np.cos(half_angle)  # qw
            ]
        
        # Set up animation in a separate thread to keep UI responsive
        def animation_thread():
            try:
                # Get rotation parameters for the move
                axis, angle = self.get_move_rotation_params(move)
                
                # Get affected pieces
                affected_positions = self.get_affected_pieces(move)
                pieces_to_rotate = []
                
                # Gather affected piece meshes
                for pos in affected_positions:
                    i, j, k = pos
                    if i < 3 and j < 3 and k < 3:  # Ensure indices are in range
                        piece_id = self.cube_tracker.piece_current_positions[i, j, k]
                        if piece_id != 14 and piece_id in self.piece_meshes:
                            pieces_to_rotate.append(self.piece_meshes[piece_id]['mesh'])
                
                # Create a temporary group for rotation
                rotation_group = Group()
                self.scene.add(rotation_group)
                
                # Move pieces to the rotation group
                for piece in pieces_to_rotate:
                    if piece in self.cube_group.children:
                        self.cube_group.remove(piece)
                    rotation_group.add(piece)
                
                # Animate the rotation using quaternions
                steps = 10
                for step in range(steps + 1):
                    rotation_angle = angle * step / steps
                    quaternion = axis_angle_to_quaternion(axis, rotation_angle)
                    rotation_group.quaternion = quaternion
                    time.sleep(self.animation_speed / steps)
                
                # Apply the move to the tracker
                self.cube_tracker.apply_moves(move)
                
                # Move pieces back to the cube group
                for piece in list(rotation_group.children):
                    rotation_group.remove(piece)
                    self.cube_group.add(piece)
                
                # Clean up
                self.scene.remove(rotation_group)
                self.update_visualization()
            except Exception as e:
                with self.output:
                    print(f"Animation error: {e}")
            finally:
                self.currently_animating = False
        
        # Start animation thread
        threading.Thread(target=animation_thread).start()
    
    def update_history_display(self):
        """Update the move history display"""
        formatted_history = []
        for move in self.move_history:
            if move.islower():
                formatted_history.append(f"{move.upper()}'")
            else:
                formatted_history.append(move)
        
        history_str = " ".join(formatted_history[-20:])  # Show last 20 moves
        with self.output:
            self.history_display.value = f"<b>Move History:</b> {history_str}"
    
    def create_ui_controls(self):
        """Create UI controls for interacting with the cube"""
        # Move buttons
        self.move_buttons = {}
        button_layout = Layout(width='50px', margin='2px')
        
        for move in ['L', 'R', 'F', 'B', 'U', 'D']:
            # Clockwise button
            cw_button = Button(description=move, layout=button_layout, 
                               button_style='primary')
            cw_button.on_click(lambda b, m=move: self.handle_move(m))
            
            # Counter-clockwise button
            ccw_button = Button(description=f"{move}'", layout=button_layout, 
                               button_style='info')
            ccw_button.on_click(lambda b, m=move.lower(): self.handle_move(m))
            
            self.move_buttons[move] = (cw_button, ccw_button)
        
        # Utility buttons
        self.scramble_button = Button(
            description='Scramble', 
            layout=Layout(width='100px', margin='2px'),
            button_style='warning'
        )
        self.scramble_button.on_click(self.handle_scramble)
        
        self.reset_button = Button(
            description='Reset', 
            layout=Layout(width='100px', margin='2px'),
            button_style='danger'
        )
        self.reset_button.on_click(self.handle_reset)
        
        # Algorithm sequence buttons
        self.sequence_buttons = {}
        seq_layout = Layout(width='100px', margin='2px')
        
        common_sequences = {
            'Sexy Move': 'RUru',
            'T Perm': 'RUrFRRuRUFRF',
            'Y Perm': 'FRUruRFrFUrFRuRF'
        }
        
        for name, sequence in common_sequences.items():
            btn = Button(description=name, layout=seq_layout, button_style='success')
            btn.on_click(lambda b, seq=sequence: self.handle_sequence(seq))
            self.sequence_buttons[name] = btn
        
        # Animation speed control
        self.speed_slider = FloatSlider(
            value=0.3,
            min=0.1,
            max=1.0,
            step=0.1,
            description='Speed:',
            continuous_update=False,
            layout=Layout(width='250px')
        )
        self.speed_slider.observe(self.update_speed, names='value')
        
        # Move history display
        self.history_display = HTML(value="<b>Move History:</b>")
        
        # Arrange UI controls
        basic_moves = [
            HBox([self.move_buttons[m][0], self.move_buttons[m][1]]) 
            for m in ['L', 'R', 'F', 'B', 'U', 'D']
        ]
        
        sequences = HBox([self.sequence_buttons[name] for name in common_sequences])
        utilities = HBox([self.scramble_button, self.reset_button])
        
        # Main control panel
        self.controls = VBox([
            HTML("<h3>Rubik's Cube Simulator</h3>"),
            HTML("<b>Basic Moves:</b>"),
            HBox([VBox(basic_moves[:2]), VBox(basic_moves[2:4]), VBox(basic_moves[4:])]),
            HTML("<b>Common Algorithms:</b>"),
            sequences,
            HTML("<b>Utilities:</b>"),
            utilities,
            self.speed_slider,
            self.history_display,
            self.output
        ])
    
    def update_speed(self, change):
        """Update animation speed from slider"""
        self.animation_speed = change.new
    
    def handle_move(self, move):
        """Handle a move button click"""
        if not self.currently_animating:
            self.animate_move(move)
    
    def handle_sequence(self, sequence):
        """Apply a sequence of moves"""
        if self.currently_animating:
            return
        
        def sequence_thread():
            for move in sequence:
                self.handle_move(move)
                # Wait for animation to complete
                while self.currently_animating:
                    time.sleep(0.05)
                time.sleep(0.1)  # Small pause between moves
        
        threading.Thread(target=sequence_thread).start()
    
    def handle_scramble(self, b):
        """Scramble the cube with random moves"""
        if self.currently_animating:
            return
            
        def scramble_thread():
            try:
                # Generate random sequence of 20 moves
                possible_moves = list('LRFBUDlrfbud')
                moves = [np.random.choice(possible_moves) for _ in range(20)]
                
                with self.output:
                    print(f"Scrambling with sequence: {''.join(moves)}")
                
                for move in moves:
                    # Wait if currently animating
                    while self.currently_animating:
                        time.sleep(0.1)
                    
                    # Apply next move
                    self.animate_move(move)
                    
                    # Wait for animation to complete
                    time.sleep(self.animation_speed + 0.1)
            except Exception as e:
                with self.output:
                    print(f"Scramble error: {e}")
        
        threading.Thread(target=scramble_thread).start()
    
    def handle_reset(self, b):
        """Reset the cube to solved state"""
        if self.currently_animating:
            return
        
        # Create a new cube tracker to reset to initial state
        self.cube_tracker = CubeTracker()
        self.move_history = []
        self.update_history_display()
        
        # Update visualization
        self.create_cube_pieces()
    
    def display(self):
        """Return the widget for display"""
        return VBox([
            self.renderer,
            self.controls
        ])


def create_rubiks_cube_visualization():
    """Create and return the Rubik's cube visualization widget"""
    cube_tracker = CubeTracker()
    visualizer = CubeVisualizer(cube_tracker)
    return visualizer.display()

# Display the interactive Rubik's cube widget
cube_widget = create_rubiks_cube_visualization()
display(cube_widget)